# BeautifulSoup4를 이용한 크롤링

# BeautifulSoup4, Selenium으로 크롤링하기

In [1]:
%pip install beautifulsoup4 requests fake-useragent

Note: you may need to restart the kernel to use updated packages.


## 정적 페이지 가지고 놀기

### 1. 페이지를 얻어오기

In [2]:
import requests

# requests.get은 페이지를 가져오는 함수이다.
response = requests.get("https://www.google.com")

# Status Code가 200이면 페이지를 가져오는 것에 성공한 것이다.
print ("Response code:", response.status_code)

# response.text는 받아온 것을 텍스트로 가져온다
print ("Response page:", response.text[0:200], "\n...")

# response.url은 최종적으로 어느 URL의 페이지를 가져왔는지 저장한다.
print ("Response URL:", response.url)

# response.json()은 받아온 것이 json이면 데이터를 파이썬의 사전형 데이터로 변환한다.
# print ("Response json:", response.json())

# 가장 날것인 상태는 response.content이다.

Response code: 200
Response page: <!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/goo 
...
Response URL: https://www.google.com/


In [3]:
import requests

# 몇몇 페이지는 requests에 User Agent가 정상적이지 않아 보이면 요청을 거부해버린다
response = requests.get("https://namu.wiki/w/")
print ("Response code:", response.status_code)

Response code: 403


In [4]:
# 몇몇 페이지는 requests에 User Agent가 정상적이지 않아 보이면 연결을 끊어버린다
try:
  requests.get("https://dictionary.cambridge.org/")
except Exception as e:
  print (e)

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [5]:
from fake_useragent import UserAgent

# 페이지 요청을 다른 브라우저에서 요청한 척 하게 만들어줄 때 쓸 객체
ua = UserAgent()

# 헤더의 User-Agent를 지정해서 크롬 브라우저에서 요청한 척 하면 페이지를 받아올 수 있다.
# 더 많은 user agent는 https://github.com/hellysmile/fake-useragent 에서 확인 가능하다.
response = requests.get("https://namu.wiki/", headers={
  "User-Agent": ua.chrome
})

In [6]:
# 주의: 몇몇 페이지는 모바일 User Agent를 쓰면 요청을 모바일 페이지로 Redirect할 수도 있다.
response = requests.get("https://novel.naver.com/webnovel/weekday", headers={
  "User-Agent": "Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25"
})

# mobile page로 redirect된 것을 확인 가능하다.
response.url

'https://m.novel.naver.com/webnovel/weekday'

### 2. 페이지를 수프로 만들어 추출 준비하기

In [7]:
# 페이지 가져오기
from fake_useragent import UserAgent

ua = UserAgent()
response = requests.get("https://dictionary.cambridge.org/dictionary/english/respect", headers={
  "User-Agent": ua.chrome
})

page = response.text

In [8]:
from bs4 import BeautifulSoup

# BeautifulSoup에 요청으로 가져온 페이지를 먹인다.
# 결과로 나온 soup는 기본적으로 Document 전체를 나타내는 객체이다.
page_soup = BeautifulSoup(page, 'html.parser')

In [9]:
# 잘 들어간 것을 확인해보기
print(str(page_soup.prettify())[0:200])

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="/common.css?version=5.0.243" rel="stylesheet" type="text/css"/>
  <style>
   .i-amphtml-element>[overflow] {
           display: block !important


### 3. 수프에서 필요한 데이터를 추출하기

In [10]:
# 페이지를 먼저 눈으로 보면서 분석을 해보자.
with open("dictionary.html", 'w') as fp:
  fp.write(page_soup.prettify())

#### find_all, find로 원하는 데이터를 담은 soup 찾기

In [11]:
# soup의 find_all은 조건에 맞는 모든 하위 soup를 찾는다.
meta_soup_list = page_soup.find_all('meta')

meta_soup_list[0:5]

[<meta charset="utf-8"/>,
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
 <meta content="respect definition: 1. admiration felt or shown for someone or something that you believe has good ideas or qualities…. Learn more." name="description"/>
 <meta content="respect definition, dictionary, english, british, american, business, british english, thesaurus, define respect, respect meaning, what is respect, spelling, conjugation, audio pronunciation, free, online, english." name="keywords"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
 <script charset="UTF-8" type="text/javascript">
             window.geofeed = function(geoFeedData) {
                 window.geofeedData = geoFeedData;
 
                 window.isGDPRMode = function() {
                     return "false" === "true";
                 };
 
                 window.isCCPAMode = function() {
                 

In [12]:
# 개수에 제한을 걸 수 있다.
meta_soup_limited_list = page_soup.find_all('a', limit=5)
meta_soup_limited_list

[<a class="hdib lpb-5 lpt-1" href="/" title="Cambridge Dictionary">
 <amp-img alt="Cambridge Dictionary" height="30" noloading="" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"></amp-img>
 <noscript>
 <img alt="Cambridge Dictionary" class="lpb-5" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"/>
 </noscript>
 </a>,
 <a class="hdb lpt-10 lpb-10 lmr-25 vh-a" href="/dictionary/"><span class="hdib lpt-2">Dictionary</span></a>,
 <a class="hdb hao lpt-10 lpb-10 lmr-25" href="/translate/"><span class="hdib lpt-2">Translate</span></a>,
 <a class="hdb hao lpt-10 lpb-10 lmr-25" href="/grammar/british-grammar/"><span class="hdib lpt-2">Grammar</span></a>,
 <a class="hdb hao lpt-10 lpb-10 lmr-25" href="/thesaurus/"><span class="hdib lpt-2">Thesaurus</span></a>]

In [13]:
# find_all의 recursive 인수에 False를 넣으면 직계가 아닌, 손자 soup들은 찾지 않는다.

all_div_limited_list = page_soup.find_all('a', limit=5, recursive=False)
all_div_limited_list

[]

In [14]:
# find는 find_all의 가장 첫번쨰 한개 찾기 버전이다.
a_soup = page_soup.find('a')
a_soup

<a class="hdib lpb-5 lpt-1" href="/" title="Cambridge Dictionary">
<amp-img alt="Cambridge Dictionary" height="30" noloading="" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"></amp-img>
<noscript>
<img alt="Cambridge Dictionary" class="lpb-5" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"/>
</noscript>
</a>

In [15]:
# find가 결과를 찾지 못하면 None을 리턴한다.
fail_soup = page_soup.find('tbody')
fail_soup

In [16]:
# find 대신 태그명들을 다음처럼 참조해서 조건을 맞춘 첫 번째 soup를 얻을 수도 있다.
page_soup.header.div.div.div

<div class="hfl">
<div class="hdib hv-3 lpt-15 lpl-15 lpr-15 lp-l_l-25">
<span aria-label="Open site navigation panel" class="cb hao lpt-2 hp" on="tap:AMP.setState({ stateSearch: { autocomplete: false } }), sidebarNav.open" role="button" tabindex="0"><i></i></span>
</div>
<div class="hdib hvt hao tc-bd lpt-10 lpb-2 lpr-15 lbr-s lb-ch">
<a class="hdib lpb-5 lpt-1" href="/" title="Cambridge Dictionary">
<amp-img alt="Cambridge Dictionary" height="30" noloading="" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"></amp-img>
<noscript>
<img alt="Cambridge Dictionary" class="lpb-5" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"/>
</noscript>
</a>
</div>
</div>

#### find_parents, find_parent, find_next_siblings, find_next_sibling, find_prev_sibling, find_prev_siblings

In [17]:
# 임의의 soup 하나를 받아옴
some_soup = page_soup.html.body.header.div.div.div
print (some_soup.prettify())

<div class="hfl">
 <div class="hdib hv-3 lpt-15 lpl-15 lpr-15 lp-l_l-25">
  <span aria-label="Open site navigation panel" class="cb hao lpt-2 hp" on="tap:AMP.setState({ stateSearch: { autocomplete: false } }), sidebarNav.open" role="button" tabindex="0">
   <i>
   </i>
  </span>
 </div>
 <div class="hdib hvt hao tc-bd lpt-10 lpb-2 lpr-15 lbr-s lb-ch">
  <a class="hdib lpb-5 lpt-1" href="/" title="Cambridge Dictionary">
   <amp-img alt="Cambridge Dictionary" height="30" noloading="" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95">
   </amp-img>
   <noscript>
    <img alt="Cambridge Dictionary" class="lpb-5" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"/>
   </noscript>
  </a>
 </div>
</div>



In [18]:
# find_parents는 그 soup의 부모부터 시작해서 올라가는 족보를 전부 찾는 함수이다.
parents_soup = some_soup.find_parents()

# document 전체, html, body, header, div, div 총 6개의 soup가 결과가 된다.
len(parents_soup)

6

In [19]:
# find_parent는 그 soup의 부모만 전부 찾는 함수이다.
parent_soup = some_soup.find_parent()
print (parent_soup.prettify())

<div class="hoh flx-w_no">
 <div class="hfl">
  <div class="hdib hv-3 lpt-15 lpl-15 lpr-15 lp-l_l-25">
   <span aria-label="Open site navigation panel" class="cb hao lpt-2 hp" on="tap:AMP.setState({ stateSearch: { autocomplete: false } }), sidebarNav.open" role="button" tabindex="0">
    <i>
    </i>
   </span>
  </div>
  <div class="hdib hvt hao tc-bd lpt-10 lpb-2 lpr-15 lbr-s lb-ch">
   <a class="hdib lpb-5 lpt-1" href="/" title="Cambridge Dictionary">
    <amp-img alt="Cambridge Dictionary" height="30" noloading="" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95">
    </amp-img>
    <noscript>
     <img alt="Cambridge Dictionary" class="lpb-5" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"/>
    </noscript>
   </a>
  </div>
 </div>
 <nav class="chn hoh hdn hdb-s fs14" id="main-nav">
  <ul class="hul-u hul-u0 hax hvt tb lmb-0 lml-10">
   <li class="hdib">
    <a class="hdb lpt-10 lpb-10 lmr-25 vh-a" href="/dictionary/">
     <span

In [20]:
# find_next_siblings는 같은 부모를 공유하는 노드들 중 자신보다 뒤에 있는 노드들을 찾아낸다.
# find_next_sibling은 한개짜리 버전이다.
next_sibling_soup = some_soup.find_next_sibling()
print (next_sibling_soup.prettify())

<nav class="chn hoh hdn hdb-s fs14" id="main-nav">
 <ul class="hul-u hul-u0 hax hvt tb lmb-0 lml-10">
  <li class="hdib">
   <a class="hdb lpt-10 lpb-10 lmr-25 vh-a" href="/dictionary/">
    <span class="hdib lpt-2">
     Dictionary
    </span>
   </a>
  </li>
  <li class="hdib">
   <a class="hdb hao lpt-10 lpb-10 lmr-25" href="/translate/">
    <span class="hdib lpt-2">
     Translate
    </span>
   </a>
  </li>
  <li class="hdib">
   <a class="hdb hao lpt-10 lpb-10 lmr-25" href="/grammar/british-grammar/">
    <span class="hdib lpt-2">
     Grammar
    </span>
   </a>
  </li>
  <li class="hdib">
   <a class="hdb hao lpt-10 lpb-10 lmr-25" href="/thesaurus/">
    <span class="hdib lpt-2">
     Thesaurus
    </span>
   </a>
  </li>
  <li class="hdib">
   <a class="hdb hao lpt-10 lpb-10" href="/plus/">
    <span class="hdib lpt-2" id="plus-s">
     +Plus
    </span>
    <span class="hdn lpt-2" id="plus-w">
     Cambridge Dictionary +Plus
    </span>
   </a>
  </li>
 </ul>
</nav>



In [21]:
# find_prev_siblings는 같은 부모를 공유하는 노드들 중 자신보다 전에 있는 노드들을 찾아낸다.
# find_prev_sibling은 한개짜리 버전이다.

prev_sibling_soups = some_soup.find_previous_siblings()
print (prev_sibling_soups)

[]


#### attributes를 이용해서 soup에서 찾기

In [22]:
# 각 노드가 가지는 id, class, href 등을 우리는 "attributes"라고 부르고, attrs로 줄여쓴다.
page_soup.a.attrs


{'class': ['hdib', 'lpb-5', 'lpt-1'],
 'href': '/',
 'title': 'Cambridge Dictionary'}

In [23]:
# soup의 특정 attribute는 다음 함수로 찾을 수 있다.
page_soup.a.get_attribute_list('href')

['/']

In [24]:
# find에 응용하여 특정 attributes를 가지는 것을 다음 방법으로 찾을 수 있다.
page_soup.find_all(attrs={ "id": "main-nav" })

[<nav class="chn hoh hdn hdb-s fs14" id="main-nav">
 <ul class="hul-u hul-u0 hax hvt tb lmb-0 lml-10">
 <li class="hdib"><a class="hdb lpt-10 lpb-10 lmr-25 vh-a" href="/dictionary/"><span class="hdib lpt-2">Dictionary</span></a></li>
 <li class="hdib"><a class="hdb hao lpt-10 lpb-10 lmr-25" href="/translate/"><span class="hdib lpt-2">Translate</span></a></li>
 <li class="hdib"><a class="hdb hao lpt-10 lpb-10 lmr-25" href="/grammar/british-grammar/"><span class="hdib lpt-2">Grammar</span></a></li>
 <li class="hdib"><a class="hdb hao lpt-10 lpb-10 lmr-25" href="/thesaurus/"><span class="hdib lpt-2">Thesaurus</span></a></li>
 <li class="hdib">
 <a class="hdb hao lpt-10 lpb-10" href="/plus/">
 <span class="hdib lpt-2" id="plus-s">+Plus</span>
 <span class="hdn lpt-2" id="plus-w">Cambridge Dictionary +Plus</span>
 </a>
 </li>
 </ul>
 </nav>]

In [25]:
page_soup.find_all(attrs={ "src": "/external/images/logo-lrg-small.png?version=5.0.243" })

[<amp-img alt="Cambridge Dictionary" height="30" noloading="" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"></amp-img>,
 <img alt="Cambridge Dictionary" class="lpb-5" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"/>,
 <amp-img alt="Cambridge Dictionary" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"></amp-img>,
 <img alt="Cambridge Dictionary" class="lpb-5" height="30" src="/external/images/logo-lrg-small.png?version=5.0.243" width="95"/>]

In [26]:
# attrs를 직접 인수로 넣어도 같은 동작을 한다.
page_soup.find(id='main-nav')

<nav class="chn hoh hdn hdb-s fs14" id="main-nav">
<ul class="hul-u hul-u0 hax hvt tb lmb-0 lml-10">
<li class="hdib"><a class="hdb lpt-10 lpb-10 lmr-25 vh-a" href="/dictionary/"><span class="hdib lpt-2">Dictionary</span></a></li>
<li class="hdib"><a class="hdb hao lpt-10 lpb-10 lmr-25" href="/translate/"><span class="hdib lpt-2">Translate</span></a></li>
<li class="hdib"><a class="hdb hao lpt-10 lpb-10 lmr-25" href="/grammar/british-grammar/"><span class="hdib lpt-2">Grammar</span></a></li>
<li class="hdib"><a class="hdb hao lpt-10 lpb-10 lmr-25" href="/thesaurus/"><span class="hdib lpt-2">Thesaurus</span></a></li>
<li class="hdib">
<a class="hdb hao lpt-10 lpb-10" href="/plus/">
<span class="hdib lpt-2" id="plus-s">+Plus</span>
<span class="hdn lpt-2" id="plus-w">Cambridge Dictionary +Plus</span>
</a>
</li>
</ul>
</nav>

In [27]:
# class의 경우 python의 예약어이기 때문에 특별히 다음과 같이 다른 인수를 사용한다.
page_soup.find(class_='dhw')

<span class="headword hdb tw-bw dhw dpos-h_hw"><span class="hw dhw">respect</span></span>

#### select를 사용하여 원하는 데이터를 담은 soup 찾기

In [28]:
# select는 CSS Selector 방식으로 soup를 찾는 함수이다.
# 참고: https://www.w3schools.com/cssref/css_selectors.asp
need_soups = page_soup.select('div.di-body')

print (len(need_soups))

3


In [29]:
# select_one 함수는 select의 한개 찾기 버전이다.
need_soup = page_soup.select_one('div.di-body')

print (need_soup.prettify()[0:400], "\n...")

<div class="di-body">
 <div class="entry">
  <div class="entry-body">
   <div class="pr entry-body__el">
    <div class="cid" id="cald4-1">
    </div>
    <div class="pos-header dpos-h">
     <div class="di-title">
      <span class="headword hdb tw-bw dhw dpos-h_hw">
       <span class="hw dhw">
        respect
       </span>
      </span>
     </div>
     <div class="posgram dpos-g hdib lmr-5">
 
...


In [30]:
# 원하는 데이터가 담긴 soup를 다음처럼 찾을 수 있다.
page_soup.select("html > body div.di-body span.hw.dhw")

[<span class="hw dhw">respect</span>,
 <span class="hw dhw">respect</span>,
 <span class="hw dhw">respect</span>,
 <span class="hw dhw">respect</span>,
 <span class="hw dhw">respect</span>]

#### 찾은 soup에서 데이터를 구하기

In [31]:
# text는 soup 내부에 문자열이 있으면 그것들을 모두 적절히 합쳐서 리턴한다.
page_soup.find('span', class_='dhw').text

'respect'

In [32]:
some_fool_html_string = """\
<div>\
Some\
<br />\
Plain\
<a href="/foo">Link</a>\
Text\
<div><p>Children</p></div>\
<div>\
"""

print ("HTML:", some_fool_html_string)

some_awesome_soup = BeautifulSoup(some_fool_html_string)

some_awesome_soup.text

HTML: <div>Some<br />Plain<a href="/foo">Link</a>Text<div><p>Children</p></div><div>


'SomePlainLinkTextChildren'

In [33]:
# href, option의 name 등은 attributes를 통해 얻는다.

print("attrs 직접 참조:", page_soup.a.attrs["href"])
print("생략 가능:", page_soup.a["href"])

attrs 직접 참조: /
생략 가능: /


In [34]:
# 이 기능들을 사용해서, 다음과 같이 원하는 데이터들을 구할 수 있다.

main_body_soup = page_soup.select_one('div.di-body')

def try_get_text(soup):
  try:
    return soup.text.strip()
  except:
    return None


try:
# noun, verb, ...
  data_mean_set = []
  for one_part_soup in main_body_soup.find_all("div", class_="entry-body__el"):
    # pos-header
    header_soup = one_part_soup.select_one("div.pos-header")

    # respect
    data_header_word = try_get_text( header_soup.select_one("div.di-title") )
    # noun, verb [T]
    data_header_part = try_get_text( header_soup.select_one("div.posgram") )

    
    for one_mean_soup in one_part_soup.select('div.pr.dsense'):
      data_word = try_get_text( one_mean_soup.find('span', class_='dsense_hw') )
      data_part = try_get_text( one_mean_soup.find('span', class_='dsense_pos') )
      data_guide_word = try_get_text( one_mean_soup.find('span', class_='dsense_gw') )

      data_definition = try_get_text ( one_mean_soup.select_one('div.def') )

      data_mean_set.append({
        "data_header_word": data_header_word,
        "data_header_part": data_header_part,
        "word": data_word,
        "part": data_part,
        "guide": data_guide_word,
        "definition": data_definition
      })

except Exception as e:
  print (e)


for value in data_mean_set:
  print (value)

{'data_header_word': 'respect', 'data_header_part': 'noun', 'word': 'respect', 'part': 'noun', 'guide': '(ADMIRATION)', 'definition': 'admiration felt or shown for someone or something that you believe has good ideas or qualities:'}
{'data_header_word': 'respect', 'data_header_part': 'noun', 'word': 'respect', 'part': 'noun', 'guide': '(HONOUR)', 'definition': 'politeness, honour, and care shown towards someone or something that is considered important:'}
{'data_header_word': 'respect', 'data_header_part': 'noun', 'word': 'respect', 'part': 'noun', 'guide': '(FEATURE)', 'definition': 'a particular feature or detail:'}
{'data_header_word': 'respect', 'data_header_part': 'verb [ T ]', 'word': 'respect', 'part': 'verb', 'guide': '(ADMIRE)', 'definition': 'to feel or show admiration for someone or something that you believe has good ideas or qualities:'}
{'data_header_word': 'respect', 'data_header_part': 'verb [ T ]', 'word': 'respect', 'part': 'verb', 'guide': '(HONOUR)', 'definition': '

### 4. 얻은 데이터를 저장하기

파이썬의 변수에만 데이터가 들어있는 상태는 프로그램이 꺼지면 날라가는 휘발성 상태이다.

DB 또는 파일에 저장하는 식으로 영구히 데이터를 보존할 수 있다.

## 동적 페이지 가지고 놀기

In [35]:
%pip install selenium chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


### 1. 동적 페이지를 가져와서 정적 페이지처럼 만들기

In [36]:
import os
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.chrome.service import Service


# 크롬 드라이버를 설치 후 그 위치를 리턴하는 함수이다.
chrome_path = chromedriver_autoinstaller.install(True)

# 크롬 드라이버를 셀레니움이 제공하는 서비스라는 것으로 감싸준다.
s = Service(chrome_path)

# Selenium에게 드라이버를 먹이면 동적 페이지를 정적 페이지로 만들 수 있다.
driver = webdriver.Chrome(
  service=s, # 서비스
)

In [37]:
driver.get("https://www.google.com")
# 5초를 기다려서 크롬이 동적 페이지를 계산할 때까지 기다려 준다.
driver.implicitly_wait(5)

# 참고: 시간을 지정하는 대신 필요한 페이지의 특정 부분이 나타날때까지 기다릴 수도 있다.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# timeout 10초로 tag(tbody), id(volumeList)를 가진 node 밑에 tag(td), class(subj)를 가진 node가 나타날 때까지
driver.get("https://series.naver.com/comic/detail.series?productNo=3189295")
result = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody#volumeList td.subj")))


### 2. 가져온 페이지를 수프로 만들거나 Selenium을 써서 추출 준비하기

In [38]:
from bs4 import BeautifulSoup
# 기본적으로 Selenium은 BeautifulSoup같이 페이지에서 값을 추출하는 기능을 제공한다.

# 이 과정은 BeautifulSoup만 쓸 줄 안다 싶을때만 하면 된다.
novel_soup = BeautifulSoup(driver.find_element(By.XPATH, "//*").get_attribute('innerHTML').encode('utf-8'))
print(novel_soup.prettify()[0:500], "\n...")

<div style="display: none;">
 <input name="jindo1654180394782" type="input"/>
</div>
<head>
 <meta charset="utf-8"/>
 <meta content="-1" http-equiv="Expires"/>
 <meta content="no-cache" http-equiv="Pragma"/>
 <meta content="No-Cache" http-equiv="Cache-Control"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <title>
  네이버 시리즈
 </title>
 <link href="https://ssl.pstatic.net/static/nstore/series_favicon_152.ico" rel="shortcut icon" type="image/x-icon"/>
 <link href="https://ssl.pstatic.ne 
...


### 3. 필요한 데이터를 Selenium으로 추출하기

#### find_element 함수의 이용

In [39]:
from selenium.webdriver.common.by import By

# find_element함수는 다양한 조건으로 원하는 노드로 찾아갈 수 있다.
driver.find_element(By.CLASS_NAME, 'subj').text

'이말년씨리즈 2018 1화 가위바위보 올림픽 (2018.03.15.)'

In [40]:
# 조건을 만족하는 모든 노드를 찾기 위해 find_elements 함수를 이용한다.

for elem in driver.find_elements(By.CLASS_NAME, 'subj'):
  print (elem.text)

이말년씨리즈 2018 1화 가위바위보 올림픽 (2018.03.15.)
이말년씨리즈 2018 2화 요즘 아빠 지긋지긋해 (2018.03.15.)
이말년씨리즈 2018 3화 헬스사각지대1 (2018.03.15.)
이말년씨리즈 2018 4화 헬스사각지대2 (2018.03.15.)
이말년씨리즈 2018 5화 -단독공개- 이말년의 아이디어 짜는 법 만... (2018.03.15.)
이말년씨리즈 2018 6화 본격 이혼식 만화 1 (2018.03.15.)
이말년씨리즈 2018 7화 본격 이혼식 만화 2 (2018.03.15.)
이말년씨리즈 2018 8화 본격 이혼식 만화 3 (2018.03.15.)
이말년씨리즈 2018 9화 본격 이혼식 만화 4 (2018.03.15.)
이말년씨리즈 2018 10화 본격 이혼식 만화 5(완) (2018.03.15.)
이말년씨리즈 2018 11화 하루종일 놀 순 없어 (2018.03.15.)
이말년씨리즈 2018 12화 필살!애견에티켓 (2018.03.15.)
이말년씨리즈 2018 13화 만화영화 보러가는 날 (2018.03.15.)
이말년씨리즈 2018 14화 헬스 매니아 1 (2018.03.15.)
이말년씨리즈 2018 15화 헬스 매니아 2 (완) (2018.03.15.)
이말년씨리즈 2018 16화 솔방울 미술관 사건 1 (2018.03.15.)
이말년씨리즈 2018 17화 솔방울 미술관 사건 2 (2018.03.14.)
이말년씨리즈 2018 18화 솔방울 미술관 사건 3(완) (2018.03.14.)
이말년씨리즈 2018 19화 대머리 특별금지법 1 (2018.03.14.)
이말년씨리즈 2018 20화 대머리 특별금지법 2 (2018.03.14.)
이말년씨리즈 2018 21화 대머리 특별금지법 3 (2018.03.14.)
이말년씨리즈 2018 22화 대머리 특별금지법 4 (2018.03.16.)
이말년씨리즈 2018 23화 대머리 특별금지법 5 (2018.03.21.)
이말년씨리즈 2018 24화 대머리 특별금지법 6 (2018.

#### CSS Selector 사용

In [41]:
# find_element 함수로 CSS Selector 방식으로 값이 담긴 노드를 찾을 수 있다.
driver.find_element(By.CSS_SELECTOR, 'td.subj').text

'이말년씨리즈 2018 1화 가위바위보 올림픽 (2018.03.15.)'

In [42]:
driver.find_element(By.CSS_SELECTOR, "a[href^='#']").text

'메인메뉴 바로가기'

#### XPath 사용

In [43]:
# find_element 함수로 XPath 방식으로 값이 담긴 노드를 찾을 수 있다.
driver.find_element(By.XPATH, "//td[contains(@class, 'subj')]").text

'이말년씨리즈 2018 1화 가위바위보 올림픽 (2018.03.15.)'

In [44]:
# 복잡한 조건도 넣을 수 있다.
driver.find_element(By.XPATH, "//div[@id='content']/div[contains(@class, 'end_head')]/h2").text

'이말년씨리즈 2018'

### 4. 얻은 데이터를 저장하기

어느 방식이든 잊지 말고 데이터를 저장하도록 하자.

In [45]:
# 모든 것이 끝나면 다음 방식으로 드라이버를 닫는다.

driver.close()

## 큰 페이지에서 작은 페이지 순회하여 모든 데이터 끌어오기

## 외전: 자동화된 프로그램에서의 셀레니움

In [46]:
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


# 크롬 드라이버를 설치 후 그 위치를 리턴하는 함수이다.
chrome_path = chromedriver_autoinstaller.install(True)

# 크롬 드라이버를 셀레니움이 제공하는 서비스라는 것으로 감싸준다.
s = Service(chrome_path)


# 웹 드라이버 옵션
driver_options = Options()

# 자동으로 나오는 브라우저가 안열리게 하는 기능
driver_options.add_argument('--headless')

# 보이지 않는 브라우저가 실제로 어떤 크기를 가지는지 지정 (너비, 높이)
driver_options.add_argument("--window-size=1920,1080")

# 화면이 안나오는 환경에서 이미지 다운로드를 막아서 너무 많은 리소스를 크롤링때 데려오는걸 막음
driver_options.add_argument("--blink-settings=imagesEnabled=false")
driver_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2})

# 화면이 나타나지 않는 리눅스에서 추가해야하는 기능
driver_options.add_argument('--no-sandbox')


driver = webdriver.Chrome(
  service=s, # 서비스
  chrome_options=driver_options, # 추가 옵션
)

/var/folders/8w/jy24r__500q81wl60g2fqld00000gn/T/ipykernel_32344/4282400188.py:32: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


In [47]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 적당한 페이지를 가져왔을 때,
driver.get("https://series.naver.com/comic/detail.series?productNo=3189295")
result = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody#volumeList td.subj")))

# 크롬창이 없더라도 페이지를 제대로 잘 가져왔는지 다음 명령으로 스크린샷을 떠서 확인할 수 있다.
with open ("screenshot.png", 'wb') as fp:
  fp.write(driver.get_screenshot_as_png())


driver.close()